## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Puerto Ayora,EC,2022-06-08 21:22:48,-0.7393,-90.3518,76.96,88,53,5.99
1,1,Langxiang,CN,2022-06-08 21:22:48,46.9500,128.8833,56.62,92,66,2.44
2,2,Ajaccio,FR,2022-06-08 21:22:49,42.0000,8.9167,69.85,68,40,4.61
3,3,Jacareacanga,BR,2022-06-08 21:22:49,-6.2222,-57.7528,81.34,71,100,2.21
4,4,Albany,US,2022-06-08 21:20:30,42.6001,-73.9662,75.79,49,23,1.01


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Puerto Ayora,EC,2022-06-08 21:22:48,-0.7393,-90.3518,76.96,88,53,5.99
2,2,Ajaccio,FR,2022-06-08 21:22:49,42.0000,8.9167,69.85,68,40,4.61
3,3,Jacareacanga,BR,2022-06-08 21:22:49,-6.2222,-57.7528,81.34,71,100,2.21
4,4,Albany,US,2022-06-08 21:20:30,42.6001,-73.9662,75.79,49,23,1.01
5,5,Bethel,US,2022-06-08 21:22:50,41.3712,-73.4140,83.01,46,0,5.75
8,8,Kapaa,US,2022-06-08 21:22:50,22.0752,-159.3190,83.48,73,0,8.05
9,9,Ixtapa,MX,2022-06-08 21:22:51,20.7000,-105.2000,79.83,78,100,10.36
10,10,Bredasdorp,ZA,2022-06-08 21:22:51,-34.5322,20.0403,60.60,76,4,10.94
13,13,Carutapera,BR,2022-06-08 21:22:52,-1.1950,-46.0200,78.03,86,76,7.11
14,14,Cidreira,BR,2022-06-08 21:22:08,-30.1811,-50.2056,62.83,85,5,9.95


In [5]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.isnull().sum()
# OR
#preferred_cities_df.notnull().sum()
# OR
preferred_cities_df.count()

City_ID       410
City          410
Country       408
Date          410
Lat           410
Lng           410
Max Temp      410
Humidity      410
Cloudiness    410
Wind Speed    410
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
clean_df.columns

Index(['City_ID', 'City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp',
       'Humidity', 'Cloudiness', 'Wind Speed'],
      dtype='object')

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.96,-0.7393,-90.3518,
2,Ajaccio,FR,69.85,42.0000,8.9167,
3,Jacareacanga,BR,81.34,-6.2222,-57.7528,
4,Albany,US,75.79,42.6001,-73.9662,
5,Bethel,US,83.01,41.3712,-73.4140,
8,Kapaa,US,83.48,22.0752,-159.3190,
9,Ixtapa,MX,79.83,20.7000,-105.2000,
10,Bredasdorp,ZA,60.60,-34.5322,20.0403,
13,Carutapera,BR,78.03,-1.1950,-46.0200,
14,Cidreira,BR,62.83,-30.1811,-50.2056,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
2,Ajaccio,FR,69.85,42.0000,8.9167,Sole E Monte
3,Jacareacanga,BR,81.34,-6.2222,-57.7528,Casa do Ian😎
4,Albany,US,75.79,42.6001,-73.9662,
5,Bethel,US,83.01,41.3712,-73.4140,Courtyard by Marriott Danbury
8,Kapaa,US,83.48,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Ixtapa,MX,79.83,20.7000,-105.2000,hotel la terminal
10,Bredasdorp,ZA,60.60,-34.5322,20.0403,Bredasdorp Country Manor
13,Carutapera,BR,78.03,-1.1950,-46.0200,Lidera
14,Cidreira,BR,62.83,-30.1811,-50.2056,Hotel Castelo


In [13]:
# 8a. Create the output File (CSV)
output_file = "Hotel_data/Hotels.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))